In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer,AutoModel
import numpy as np

In [12]:
import yfinance as yf

In [17]:
# Fetch Nifty-50 and VIX daily
nifty_data = yf.download('^NSEI', start='2020-01-01', end='2026-01-26')
vix_data = yf.download('^INDIAVIX', start='2020-01-01', end='2026-01-26')

# Combine into single DataFrame
market_df = pd.concat([nifty_data['Close'], vix_data['Close']], axis=1, keys=['nifty_close', 'vix'])
market_df = market_df.bfill()
# 3D features exactly as Nifty50 report
market_df['returns'] = market_df['nifty_close'].pct_change()
market_df['log_price_norm'] = np.log(market_df['nifty_close']) - np.log(market_df['nifty_close']).median()
market_df['vix_norm'] = (market_df['vix'] - market_df['vix'].mean()) / market_df['vix'].std()


/tmp/ipython-input-470651841.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty_data = yf.download('^NSEI', start='2020-01-01', end='2026-01-26')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-470651841.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix_data = yf.download('^INDIAVIX', start='2020-01-01', end='2026-01-26')
[*********************100%***********************]  1 of 1 completed


In [18]:
market_df


,nifty_close,vix,returns,log_price_norm,vix_norm
Ticker,^NSEI,^INDIAVIX,,,
Date,,,,,
2020-01-01,12182.500000,11.49,NaN,-0.391921,-0.766396
2020-01-02,12282.200195,11.49,0.008184,-0.383771,-0.766396
2020-01-03,12226.650391,12.70,-0.004523,-0.388304,-0.609439
2020-01-06,11993.049805,14.78,-0.019106,-0.407595,-0.339630
2020-01-07,12052.950195,14.61,0.004995,-0.402612,-0.361681
...,...,...,...,...,...
2026-01-19,25585.500000,11.83,-0.004236,0.350104,-0.722293
2026-01-20,25232.500000,12.73,-0.013797,0.336211,-0.605548


In [ ]:
df_csv=pd.read_csv('../data/data_with_embeddings.csv')

(2192, 1026)

In [33]:
market_df_reset = market_df.reset_index()

In [66]:
market_df_selected = market_df_reset[['Date','returns','log_price_norm','vix_norm']]

In [67]:
market_df_selected.columns =['Date','returns','log_price_norm','vix_norm']

In [68]:
market_df_selected

,Date,returns,log_price_norm,vix_norm
0,2020-01-01,NaN,-0.391921,-0.766396
1,2020-01-02,0.008184,-0.383771,-0.766396
2,2020-01-03,-0.004523,-0.388304,-0.609439
3,2020-01-06,-0.019106,-0.407595,-0.339630
4,2020-01-07,0.004995,-0.402612,-0.361681
...,...,...,...,...
1497,2026-01-19,-0.004236,0.350104,-0.722293
1498,2026-01-20,-0.013797,0.336211,-0.605548
1499,2026-01-21,-0.002972,0.333234,-0.469346
1500,2026-01-22,0.005263,0.338483,-0.525124


In [ ]:
df_csv


,Date,Headlines,0,1,2,3,4,5,6,7,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,2020-01-01,"['Tata Consumer reports moderate India growth,...",0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,-0.031760,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
1,2020-01-02,"['Tata Consumer reports moderate India growth,...",0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,-0.031760,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
2,2020-01-03,"['Tata Consumer reports moderate India growth,...",0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,-0.031760,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
3,2020-01-04,"['Tata Consumer reports moderate India growth,...",0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,-0.031760,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
4,2020-01-05,"['Tata Consumer reports moderate India growth,...",0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,-0.031760,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,2025-12-27,"['Form 6K HDFC BANK LTD For: 29 December', 'Fo...",0.017607,0.031379,0.028967,-0.001356,0.024873,-0.074269,-0.015974,0.009117,...,-0.107867,-0.017605,-0.052345,-0.007708,-0.038739,0.001927,0.055118,0.019961,-0.008803,0.007752
2188,2025-12-28,"['Form 6K HDFC BANK LTD For: 29 December', 'Fo...",0.017607,0.031379,0.028967,-0.001356,0.024873,-0.074269,-0.015974,0.009117,...,-0.107867,-0.017605,-0.052345,-0.007708,-0.038739,0.001927,0.055118,0.019961,-0.008803,0.007752
2189,2025-12-29,"['Form 6K HDFC BANK LTD For: 29 December', 'Fo...",0.017607,0.031379,0.028967,-0.001356,0.024873,-0.074269,-0.015974,0.009117,...,-0.107867,-0.017605,-0.052345,-0.007708,-0.038739,0.001927,0.055118,0.019961,-0.008803,0.007752
2190,2025-12-30,['Shriram Finance Ltd receives Investment Bank...,0.028505,0.007402,0.000382,0.000439,0.064748,0.033883,0.012878,-0.026181,...,-0.077495,-0.020262,-0.055211,0.035356,-0.051634,-0.000062,-0.023959,0.023970,-0.004600,-0.015048


In [61]:
df_csv['Date']=pd.to_datetime(df_csv['Date'])

In [69]:
final_df = pd.merge(market_df_selected,df_csv, on ="Date", how='inner')

In [70]:
final_df.set_index('Date',inplace=True)

In [74]:
final_df.columns

Index([       'returns', 'log_price_norm',       'vix_norm',      'Headlines',
                      0,                1,                2,                3,
                      4,                5,
       ...
                   1014,             1015,             1016,             1017,
                   1018,             1019,             1020,             1021,
                   1022,             1023],
      dtype='object', length=1028)

In [75]:
final_df.drop(final_df.columns[3],axis=1,inplace=True)

In [77]:
final_df

,returns,log_price_norm,vix_norm,0,1,2,3,4,5,6,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,NaN,-0.391921,-0.766396,0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
2020-01-02,0.008184,-0.383771,-0.766396,0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
2020-01-03,-0.004523,-0.388304,-0.609439,0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
2020-01-06,-0.019106,-0.407595,-0.339630,0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
2020-01-07,0.004995,-0.402612,-0.361681,0.030777,0.039910,0.006079,0.008057,0.031532,0.036118,-0.004619,...,-0.077281,-0.039060,-0.013637,0.042577,-0.034143,-0.028902,-0.014453,0.049759,-0.009401,-0.034821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-24,-0.001339,0.371625,-1.064744,-0.005998,0.065019,0.012296,-0.021999,0.034823,-0.038383,-0.018245,...,-0.091859,0.012114,-0.037812,0.033356,-0.020459,-0.028841,0.000726,0.020800,-0.015609,0.023271
2025-12-26,-0.003818,0.367800,-1.069932,-0.012630,0.029762,0.021607,-0.020981,0.074229,-0.038849,-0.020915,...,-0.065584,-0.011411,-0.073703,0.004481,-0.004520,-0.037231,0.014197,0.027324,0.026347,-0.021259
2025-12-29,-0.003848,0.363945,-0.995994,0.017607,0.031379,0.028967,-0.001356,0.024873,-0.074269,-0.015974,...,-0.107867,-0.017605,-0.052345,-0.007708,-0.038739,0.001927,0.055118,0.019961,-0.008803,0.007752


In [102]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from collections import deque
import random
import yfinance as yf

class DuelingDDQN(nn.Module):
    """Dueling DDQN for VaR prediction"""
    def __init__(self, state_dim=1027, action_dim=10):  # 9 quantiles: 1-99%
        super().__init__()
        self.embedding = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        # Dueling streams
        self.value_stream = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.advantage_stream = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim)
        )

    def forward(self, x):
        feat = self.embedding(x)
        value = self.value_stream(feat)
        advantage = self.advantage_stream(feat)
        return value + (advantage - advantage.mean(dim=1, keepdim=True))

class DDQNVarAgent:
    def __init__(self, state_dim=1027, action_dim=10):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.q_net = DuelingDDQN(state_dim, action_dim)
        self.target_net = DuelingDDQN(state_dim, action_dim)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=0.001)
        self.memory = deque(maxlen=5000)
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.quantile_levels = np.array([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.random() <= self.epsilon:
            return random.randrange(self.action_dim)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.q_net(state)
        return q_values.argmax(1).item()

    def replay(self, batch_size=32):
        if len(self.memory) < batch_size: return

        minibatch = random.sample(self.memory, batch_size)
        states = torch.FloatTensor(np.array([e[0] for e in minibatch]))
        actions = torch.LongTensor([e[1] for e in minibatch])
        rewards = torch.FloatTensor([e[2] for e in minibatch])
        next_states = torch.FloatTensor(np.array([e[3] for e in minibatch]))
        dones = torch.BoolTensor([e[4] for e in minibatch])

        current_q = self.q_net(states).gather(1, actions.unsqueeze(1))
        next_q = self.target_net(next_states).max(1)[0].detach()
        target_q = rewards + (0.99 * next_q * ~dones)

        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_net.parameters(), 1.0)
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def update_target(self):
        self.target_net.load_state_dict(self.q_net.state_dict())

    def predict_var(self, state, confidence=0.95):
        self.q_net.eval()
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0)
            qvals = self.q_net(state_t)
            idx = np.argmin(np.abs(self.quantile_levels - confidence))
            var_pred = abs(qvals[0, idx].item())
        self.q_net.train()
        return var_pred

# 3D Feature Engineering (No News)
def create_market_features():
    # True VaR labels (rolling 252-day 5th percentile)
    true_var = []
    col_stack = final_df.to_numpy()
    for i in range(252, len(col_stack)):
        window = final_df['returns'].iloc[i-252:i].dropna()
        true_var.append(abs(np.percentile(window, 5)))

    return col_stack[251:-1], np.array(true_var), df.index[252:]

# Training (1000 episodes as per report)
def train_ddqn():
    features, true_vars, dates = create_market_features()

    split = int(0.8 * len(features))
    train_features, test_features = features[:split], features[split:]
    train_vars, test_vars = true_vars[:split], true_vars[split:]

    agent = DDQNVarAgent(state_dim=1027, action_dim=10)

    print("Training DDQN")
    for episode in range(10000):
        episode_reward = 0

        # Single episode = full training sequence
        for i in range(len(train_features)):
            state = train_features[i]
            true_var = train_vars[i]

            action = agent.act(state)
            qvals = agent.q_net(torch.FloatTensor(state).unsqueeze(0))
            pred_var = abs(qvals[0, action].item())

            # Reward: negative MAPE
            reward = -abs((true_var - pred_var) / true_var)
            next_state = train_features[min(i+1, len(train_features)-1)]
            done = (i == len(train_features) - 1)

            agent.remember(state, action, reward, next_state, done)
            episode_reward += reward
            agent.replay()

        if episode % 1000 == 0:
            agent.update_target()
            avg_reward = episode_reward / len(train_features)
            print(f"Episode {episode}, Avg Reward: {avg_reward:.4f}, ε: {agent.epsilon:.3f}")

    return agent, test_features, test_vars, dates[split:]

# Run & Evaluate
agent, test_features, test_vars, test_dates = train_ddqn()

print("\n=== TEST RESULTS (95% VaR) ===")
mape = 0
for i, (state, true_var) in enumerate(zip(test_features[:20], test_vars[:20])):
    pred_var = agent.predict_var(state, confidence=0.95)
    error_pct = abs((true_var - pred_var) / true_var) * 100
    mape += error_pct
    print(f"{test_dates[i].date()}: True={true_var:.4f}, Pred={pred_var:.4f}, Error={error_pct:.1f}%")

print(f"\nDDQN MAPE: {mape/len(test_vars):.1f}%")
print("Expected: ~17.8% (report benchmark with news)")


Training DDQN
Episode 0, Avg Reward: -38645715.4201, ε: 0.010
Episode 100, Avg Reward: -327222199402.0566, ε: 0.010
Episode 200, Avg Reward: -327222228484.4512, ε: 0.010
Episode 300, Avg Reward: -327223494504.7645, ε: 0.010
Episode 400, Avg Reward: -327224899599.2141, ε: 0.010
Episode 500, Avg Reward: -327222302016.2563, ε: 0.010
Episode 600, Avg Reward: -327222398702.7481, ε: 0.010
Episode 700, Avg Reward: -327226199567.3374, ε: 0.010
Episode 800, Avg Reward: -327222340879.5626, ε: 0.010
Episode 900, Avg Reward: -327223488234.1541, ε: 0.010

=== TEST RESULTS (95% VaR) ===


AttributeError: 'int' object has no attribute 'date'